In [1]:
import requests
import json
import pandas as pd

In [2]:
def get_lyrics(artist, song):

    url = 'https://api.lyrics.ovh/v1/' + artist + '/' + song
    try:
        response = requests.get(url)
        json_data = json.loads(response.content)
        lyrics = json_data['lyrics']
        return lyrics
    except Exception as e:
        return 'no lyrics'

In [3]:
# Load the JSON file
with open('all.json', 'r') as file:
    json_data = json.load(file)

# Normalize the nested 'data' field
df = pd.json_normalize(
    json_data, 
    record_path='data', 
    meta=['date'], 
    errors='ignore'
)

In [4]:
df.date = pd.to_datetime(df.date)
sorted_df = df.sort_values(by = ['date','weeks_on_chart', 'this_week'], ascending = [False, False, True])
sorted_df = sorted_df.drop_duplicates(subset = ["song", "artist"], keep = 'first')
sorted_df = sorted_df.sort_values(by = ['date', 'this_week'], ascending = [False, True])

In [5]:
my_dict = dict(zip(sorted_df.artist, sorted_df.song))

In [24]:
taylor_df = sorted_df[sorted_df.artist == "Taylor Swift"]

In [26]:
taylor_df['Lyrics'] = taylor_df.apply(lambda row: get_lyrics(row['artist'], row['song']), axis=1)

C:\Users\ferad\AppData\Local\Temp\ipykernel_2144\3226950773.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taylor_df['Lyrics'] = taylor_df.apply(lambda row: get_lyrics(row['artist'], row['song']), axis=1)


In [27]:
taylor_df

,song,artist,this_week,last_week,peak_position,weeks_on_chart,date,Lyrics
344521,I Can Do It With A Broken Heart,Taylor Swift,35,37.0,3,16,2024-08-17,Paroles de la chanson I Can Do It With A Broke...
344178,Down Bad,Taylor Swift,92,82.0,2,12,2024-07-20,Paroles de la chanson Down Bad par Taylor Swif...
344083,Who's Afraid Of Little Old Me?,Taylor Swift,97,86.0,9,11,2024-07-13,"The ""Who's Who"" of ""Who's That?""\r\nIs poised ..."
343984,Guilty As Sin?,Taylor Swift,98,88.0,10,10,2024-07-06,"Drowning in the Blue Nile\r\nHe sent me ""Downt..."
343771,But Daddy I Love Him,Taylor Swift,85,66.0,7,8,2024-06-22,Paroles de la chanson But Daddy I Love Him par...
...,...,...,...,...,...,...,...,...
261486,Change,Taylor Swift,100,39.0,10,3,2008-09-13,It's just a sad picture\r\nThe final blow hits...
260575,Picture To Burn,Taylor Swift,89,82.0,28,20,2008-07-12,"State the obvious, I didn't get my perfect fan..."
260134,Our Song,Taylor Swift,48,48.0,16,36,2008-06-14,I was riding shotgun with my hair undone in th...
259735,Teardrops On My Guitar,Taylor Swift,49,48.0,13,48,2008-05-17,"Drew looks at me, I fake a smile so he won't s..."


In [ ]:
import pandas as pd
import requests
from urllib.parse import quote
import time



# Function to fetch lyrics using lyrics.ovh API
def get_lyrics(row, retries=3):
    artist = quote(row['artist'])  # URL-encode the artist's name
    title = quote(row['song'])     # URL-encode the song title
    
    url = f"https://api.lyrics.ovh/v1/{artist}/{title}"
    
    for attempt in range(retries):
        try:
            response = requests.get(url)
            
            if response.status_code == 200:  # Successfully retrieved lyrics
                data = response.json()
                return data.get('lyrics')
            elif response.status_code == 429:  # Rate limit hit
                print(f"Rate limit exceeded. Waiting before retrying... (Attempt {attempt + 1}/{retries})")
                time.sleep(5)  # Wait for 5 seconds before retrying (adjust as needed)
            else:
                # Handle other HTTP errors or no lyrics found
                print(f"Failed to retrieve lyrics: HTTP {response.status_code}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Retrying... (Attempt {attempt + 1}/{retries})")
            time.sleep(5)  # Wait before retrying

    # If all retries fail
    return None

# Apply the function to each row
merged_df['lyrics'] = df.apply(get_lyrics, axis=1)

# Save the DataFrame with lyrics
merged_df.to_csv('songs_with_lyrics.csv', index=False)

Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: HTTP 404
Failed to retrieve lyrics: H